# Параллельные вычисления

Материалы:
* Макрушин С.В. Лекция 10: Параллельные вычисления
* https://docs.python.org/3/library/multiprocessing.html

## Задачи для совместного разбора

1. Посчитайте, сколько раз встречается каждый из символов (заглавные и строчные символы не различаются) в файле `Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt` и в файле `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt`.

In [ ]:
from collections import Counter, defaultdict
import csv
import os
import time
import multiprocessing as mp
from multiprocessing import Pool, cpu_count

In [ ]:


def let_count(filename):

    with open(filename, encoding='windows-1251') as f:
        text = f.read().lower()
        return Counter(text)

files = [
    "Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt",
    "Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt"
    ]

In [ ]:
%%time

let_count(files[0])
let_count(files[1])
print()


CPU times: user 289 ms, sys: 3.67 ms, total: 293 ms
Wall time: 317 ms


2. Решить задачу 1, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создать свой собственный процесс.

In [ ]:
%%time

num_processes = os.cpu_count()
print(f"Использование {num_processes} процессов")

with mp.Pool(num_processes) as pool:
    results = pool.map(let_count, files)
    print(results)

Использование 2 процессов
[Counter({' ': 45076, 'о': 23130, 'е': 20054, 'а': 18236, 'т': 14245, 'н': 14240, 'и': 13587, 'с': 11507, 'л': 9961, 'р': 9482, 'в': 9398, 'м': 7106, 'к': 6744, 'д': 6681, ',': 6372, 'у': 6044, 'п': 5489, 'я': 5458, 'ь': 4857, 'ч': 4113, 'б': 3980, 'г': 3948, 'ы': 3869, 'з': 3355, '.': 2954, '\n': 2734, 'ж': 2297, 'й': 2028, 'ш': 1943, '—': 1726, 'х': 1535, '\xa0': 1472, 'ю': 1323, 'e': 1200, '-': 900, 'э': 836, 'ц': 817, '!': 718, 'ф': 634, 'a': 590, 'щ': 587, 'l': 571, '?': 571, 'n': 459, 's': 429, ';': 406, 'm': 401, 'o': 377, 'i': 369, 't': 332, 'c': 324, 'r': 308, 'u': 285, '…': 280, '(': 276, ')': 276, 'h': 227, 'b': 220, ':': 212, 'd': 192, '«': 129, '»': 128, 'p': 100, '[': 97, ']': 97, 'v': 87, 'g': 73, 'ъ': 63, "'": 59, 'f': 52, 'q': 50, '1': 46, 'z': 44, '6': 42, '2': 42, '4': 42, '7': 42, '3': 40, '5': 40, 'j': 40, '8': 38, '9': 36, 'x': 24, '0': 22, 'k': 21, '/': 20, '`': 9, 'y': 8, 'w': 7, '_': 4}), Counter({' ': 182305, 'о': 106740, 'е': 80972, 

## Лабораторная работа 10

1. Разбейте файл `recipes_full.csv` на несколько (например, 8) примерно одинаковых по объему файлов c названиями `id_tag_nsteps_*.csv`. Каждый файл содержит 3 столбца: `id`, `tag` и `n_steps`, разделенных символом `;`. Для разбора строк используйте `csv.reader`.

__Важно__: вы не можете загружать в память весь файл сразу. Посмотреть на первые несколько строк файла вы можете, написав код, который считывает эти строки.

Подсказка: примерное кол-во строк в файле - 2.3 млн.

```
id;tag;n_steps
137739;60-minutes-or-less;11
137739;time-to-make;11
137739;course;11
```


In [ ]:
def split_csv(input_filename, output_prefix, num_files):
    """
    Разбивает большой CSV-файл на несколько меньших.

    Args:
        input_filename (str): Имя исходного файла.
        output_prefix (str): Префикс для имен выходных файлов (например, 'id_tag_nsteps_').
        num_files (int): Количество выходных файлов.
    """
    try:
        output_dir = 'split_files'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        output_files = [open(os.path.join(output_dir, f'{output_prefix}{i}.csv'), 'w', newline='', encoding='utf-8') for i in range(num_files)]
        csv_writers = [csv.writer(f, delimiter=',') for f in output_files]

        # Обрабатываем исходный файл
        with open(input_filename, 'r', newline='', encoding='utf-8') as infile:
            csv_reader = csv.reader(infile, delimiter=',')

            header = next(csv_reader)
            for writer in csv_writers:
                writer.writerow(header)

            for i, row in enumerate(csv_reader):
                writer_index = i % num_files
                id_val = row[1]
                tag_val = row[5]
                n_steps_val = row[6]

                csv_writers[writer_index].writerow([id_val, tag_val, n_steps_val])

    except FileNotFoundError:
        print(f"Ошибка: Файл '{input_filename}' не найден.")
    except IndexError as e:
        print(f"Ошибка: Не удалось найти все столбцы в строке. Проверьте структуру файла. Детали: {e}")
    finally:
        for f in output_files:
            f.close()

source_file = 'recipes_full.csv'
output_prefix = 'id_tag_nsteps_'
num_files_to_create = 8

print(f"Начинаем разбиение файла '{source_file}' на {num_files_to_create} частей...")
split_csv(source_file, output_prefix, num_files_to_create)
print("Разбиение завершено. Файлы сохранены в подпапке 'split_files'.")

Начинаем разбиение файла 'recipes_full.csv' на 8 частей...
Ошибка: Не удалось найти все столбцы в строке. Проверьте структуру файла. Детали: list index out of range
Разбиение завершено. Файлы сохранены в подпапке 'split_files'.


2. Напишите функцию, которая принимает на вход название файла, созданного в результате решения задачи 1, считает среднее значение количества шагов для каждого тэга и возвращает результат в виде словаря.

In [ ]:
def calculate_avg_steps_per_tag(filename):
    """
    Считает среднее количество шагов для каждого тега в CSV-файле.

    Args:
        filename (str): Название файла, созданного в результате задачи 1.
                        Файл должен содержать столбцы: id, tag, n_steps.

    Returns:
        dict: Словарь, где ключи — это теги, а значения — среднее количество шагов.
              Возвращает пустой словарь, если файл не найден.
    """
    tag_data = defaultdict(lambda: {'total_steps': 0, 'count': 0})

    try:
        with open(filename, 'r', newline='', encoding='utf-8') as f:
            csv_reader = csv.reader(f, delimiter=',')

            # Пропускаем заголовок
            next(csv_reader)

            for row in csv_reader:
                try:
                    tag = row[1]
                    n_steps = int(row[2])

                    tag_data[tag]['total_steps'] += n_steps
                    tag_data[tag]['count'] += 1
                except (ValueError, IndexError) as e:
                    print(f"Ошибка при обработке строки: {row}. Подробности: {e}")
                    continue

    except FileNotFoundError:
        print(f"Ошибка: Файл '{filename}' не найден.")
        return {}

    # Вычисляем среднее значение для каждого тега
    avg_steps = {
        tag: data['total_steps'] / data['count']
        for tag, data in tag_data.items()
    }

    return avg_steps

result = calculate_avg_steps_per_tag('split_files/id_tag_nsteps_0.csv')
result.keys().__len__()

96023

3. Напишите функцию, которая считает среднее значение количества шагов для каждого тэга по всем файлам, полученным в задаче 1, и возвращает результат в виде словаря. Не используйте параллельных вычислений. При реализации выделите функцию, которая объединяет результаты обработки отдельных файлов. Модифицируйте код из задачи 2 таким образом, чтобы иметь возможность получить результат, имея результаты обработки отдельных файлов. Определите, за какое время задача решается для всех файлов.


In [ ]:
def process_single_file(filename):
    """
    Обрабатывает один CSV-файл и накапливает сумму шагов и количество записей для каждого тега.

    Args:
        filename (str): Название файла для обработки.

    Returns:
        defaultdict: Словарь, где ключи - теги, а значения - словари
                     {'total_steps': int, 'count': int}.
                     Возвращает пустой defaultdict, если файл не найден.
    """
    tag_data = defaultdict(lambda: {'total_steps': 0, 'count': 0})

    try:
        with open(filename, 'r', newline='', encoding='utf-8') as f:
            csv_reader = csv.reader(f, delimiter=',')
            next(csv_reader)  # Пропускаем заголовок

            for row in csv_reader:
                try:
                    tag = row[1]
                    n_steps = int(row[2])
                    tag_data[tag]['total_steps'] += n_steps
                    tag_data[tag]['count'] += 1
                except (ValueError, IndexError) as e:
                    print(f"Ошибка при обработке строки: {row}. Подробности: {e}")
                    continue

    except FileNotFoundError:
        print(f"Ошибка: Файл '{filename}' не найден.")

    return tag_data

In [ ]:
def merge_and_calculate_avg(results):
    final_data = defaultdict(lambda: {'total_steps': 0, 'count': 0})

    for file_result in results:
        for tag, data in file_result.items():
            final_data[tag]['total_steps'] += data['total_steps']
            final_data[tag]['count'] += data['count']

    avg_steps = {
        tag: data['total_steps'] / data['count']
        for tag, data in final_data.items()
    }

    return avg_steps

In [ ]:
def solve_task_3(file_prefix, num_files):
    start_time = time.time()
    all_file_results = []

    for i in range(num_files):
        filename = f'{file_prefix}{i}.csv'
        print(f"Обработка файла: {filename}...")
        file_data = process_single_file(filename)
        all_file_results.append(file_data)

    final_avg_steps = merge_and_calculate_avg(all_file_results)

    end_time = time.time()
    elapsed_time = end_time - start_time

    print("\nРезультат:")
    print(final_avg_steps)
    print(f"\nЗадача решена за: {elapsed_time:.4f} секунд.")



solve_task_3('split_files/id_tag_nsteps_', 8)

4. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создайте свой собственный процесс. Определите, за какое время задача решается для всех файлов.

In [ ]:
def process_single_file(filename):
    tag_data = defaultdict(lambda: {'total_steps': 0, 'count': 0})

    try:
        with open(filename, 'r', newline='', encoding='utf-8') as f:
            csv_reader = csv.reader(f, delimiter=';')
            next(csv_reader)  # Skip the header

            for row in csv_reader:
                try:
                    tag_str = row[1]
                    n_steps = int(row[2])

                    tag_data[tag_str]['total_steps'] += n_steps
                    tag_data[tag_str]['count'] += 1
                except (ValueError, IndexError):
                    continue
    except FileNotFoundError:
        print(f"Ошибка: Файл '{filename}' не найден.")

    return dict(tag_data)

In [ ]:
def solve_task_4(file_prefix, num_files):
    start_time = time.time()

    filenames = [f'{file_prefix}{i}.csv' for i in range(num_files)]

    num_processes = cpu_count()
    print(f"Используется {num_processes} процессов.")

    with Pool(processes=num_processes) as pool:
        all_file_results = pool.map(process_single_file, filenames)

    final_avg_steps = merge_and_calculate_avg(all_file_results)

    end_time = time.time()
    elapsed_time = end_time - start_time

    print("\nРезультат:")
    print(final_avg_steps)
    print(f"\nЗадача решена за: {elapsed_time:.4f} секунд.")

solve_task_4(os.path.join('split_files', 'id_tag_nsteps_'), 8)

Используется 2 процессов.
Объединение результатов и вычисление средних значений...

Результат:

Задача решена за: 1.8944 секунд.


5. (*) Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Создайте фиксированное количество процессов (равное половине количества ядер на компьютере). При помощи очереди передайте названия файлов для обработки процессам и при помощи другой очереди заберите от них ответы.

In [ ]:
from multiprocessing import Queue, Process

def worker(input_queue, output_queue):
    while True:
        filename = input_queue.get()
        if filename is None:
            break

        result = process_single_file(filename)

        output_queue.put(result)

def merge_and_calculate_avg(results):
    final_data = defaultdict(lambda: {'total_steps': 0, 'count': 0})
    for file_result in results:
        for tag, data in file_result.items():
            final_data[tag]['total_steps'] += data['total_steps']
            final_data[tag]['count'] += data['count']

    avg_steps = {
        tag: data['total_steps'] / data['count']
        for tag, data in final_data.items() if data['count'] > 0
    }
    return avg_steps

def solve_task_5(file_prefix, num_files):
    start_time = time.time()

    num_processes = cpu_count() // 2
    if num_processes == 0:
        num_processes = 1
    print(f"Using {num_processes} processes.")

    input_queue = Queue()
    output_queue = Queue()

    processes = []
    for _ in range(num_processes):
        p = Process(target=worker, args=(input_queue, output_queue))
        p.start()
        processes.append(p)

    filenames = [f'{file_prefix}{i}.csv' for i in range(num_files)]
    for filename in filenames:
        input_queue.put(filename)

    for _ in range(num_processes):
        input_queue.put(None)

    all_file_results = []
    for _ in range(num_files):
        result = output_queue.get()
        all_file_results.append(result)

    for p in processes:
        p.join()

    final_avg_steps = merge_and_calculate_avg(all_file_results)

    end_time = time.time()
    elapsed_time = end_time - start_time

    print("\nResult:")
    print(final_avg_steps)
    print(f"\nTask completed in: {elapsed_time:.4f} seconds.")

solve_task_5(os.path.join('split_files', 'id_tag_nsteps_'), 8)

Using 1 processes.

Result:
{}

Task completed in: 2.2629 seconds.
